<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#匯入行銷分大表與後卡條件資料表" data-toc-modified-id="匯入行銷分大表與後卡條件資料表-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>匯入行銷分大表與後卡條件資料表</a></span></li><li><span><a href="#檢視被貼標客戶有多少會被排除" data-toc-modified-id="檢視被貼標客戶有多少會被排除-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>檢視被貼標客戶有多少會被排除</a></span></li><li><span><a href="#將需被後卡掉的人，其貼標轉成空值" data-toc-modified-id="將需被後卡掉的人，其貼標轉成空值-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>將需被後卡掉的人，其貼標轉成空值</a></span></li><li><span><a href="#檢查格式" data-toc-modified-id="檢查格式-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>檢查格式</a></span></li><li><span><a href="#存出-後卡後的行銷分大表" data-toc-modified-id="存出-後卡後的行銷分大表-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>存出 後卡後的行銷分大表</a></span></li><li><span><a href="#分析歷程與紀錄" data-toc-modified-id="分析歷程與紀錄-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>分析歷程與紀錄</a></span><ul class="toc-item"><li><span><a href="#紀錄時間-:-20220922" data-toc-modified-id="紀錄時間-:-20220922-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>紀錄時間 : 20220922</a></span></li><li><span><a href="#紀錄時間-:-20221021" data-toc-modified-id="紀錄時間-:-20221021-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>紀錄時間 : 20221021</a></span></li><li><span><a href="#紀錄時間-:-20221121" data-toc-modified-id="紀錄時間-:-20221121-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>紀錄時間 : 20221121</a></span></li><li><span><a href="#紀錄時間-:-20221220" data-toc-modified-id="紀錄時間-:-20221220-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>紀錄時間 : 20221220</a></span></li></ul></li></ul></div>

In [ ]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
# 改變輸出顏色
from colorama import init, Fore, Back

In [ ]:
# 設定大表年月
year = '2023'
month = '01'
# 大表放置路徑
result_path = 'D:\\SYW\\1.行銷分\\MKT\\2023_01_03\\result\\'       
# 後卡條件資料放置路徑
path = 'D:\\SYW\\1.行銷分\\MKT\\2023_01_03\\後卡條件\\'

## 匯入行銷分大表與後卡條件資料表

In [ ]:
# 讀取行銷分大表
df = pd.read_csv(result_path + 'IFCUSDPDT0001_' + year + month + '_eff.csv')
df.head()

In [ ]:
df.shape

In [ ]:
# 讀取後卡條件資料表(欲排除ID)
rm_df = pd.read_csv(path + 'remove_id_20221220.csv', encoding = 'cp950').rename(columns = {'ID':'SAS_ID'})
rm_df.head()

In [ ]:
rm_df.shape
# (879975, 1)

In [ ]:
# 先檢視整體POOL總共有多少人會從大表被排掉
df.SAS_ID.isin(rm_df.SAS_ID).sum()

## 檢視被貼標客戶有多少會被排除

In [ ]:
col = df.columns[df.columns.str.contains('LEVEL')]
col

In [ ]:
# 先抓出有被貼標的人
# tag_index = [i for i in range(len(df)) if any(df.loc[i, col].notna()) ]
tag_index = ( df['HOS_MKT_LEVEL'].notna() + df['SUR_MKT_LEVEL'].notna() + df['REI_MKT_LEVEL'].notna() 
             + df['ACC_MKT_LEVEL'].notna() + df['DD_MKT_LEVEL'].notna() + df['LTC_MKT_LEVEL'].notna() 
             + df['LIFE_MKT_LEVEL'].notna() )
df_tag_cus = df[tag_index]
df_tag_cus.shape
# (3709850, 38)

In [ ]:
# check : 是否成功抓出有任一貼標客戶
df_tag_cus[col].dropna(how = 'all').shape
# (3709850, 7)

In [ ]:
# 檢視有多少行銷分貼標的人會被排掉 (過往經驗約為40萬)
print('**持有任一貼標客戶中有 {} 人將會被移去原貼標(轉為空值)**'.format(df_tag_cus.SAS_ID.isin(rm_df.SAS_ID).sum()) )
# 418277

In [ ]:
# 被排除人數有點多
# 後續應檢視多半由是哪個後卡條件造成、檢視【各險種】前10%、11-30%貼標，【各別】失去多少比例的客戶
# 後續應該回測這些被卡控掉的客戶促約率、拒保、特承率

In [ ]:
# 檢視【各險種】前10%、11-30%貼標，【各別】失去多少比例的客戶
for i in col:
    for j in range(1,4):
        # XX險前10%損失客戶
        loss = df[df[i] == j].SAS_ID.isin(rm_df.SAS_ID).sum()
        # XX險前10%客戶
        pool = sum(df[i] == j)
        print( '{} 險種貼標為 {} 客戶有 {}% 將被後卡條件排除'.format(i.split('_')[0], j, round(loss/pool*100,2)) ) 

In [ ]:
# HOS 險種貼標為 1 客戶有 11.01% 將被後卡條件排除
# HOS 險種貼標為 2 客戶有 10.5% 將被後卡條件排除
# HOS 險種貼標為 3 客戶有 10.03% 將被後卡條件排除
# SUR 險種貼標為 1 客戶有 9.93% 將被後卡條件排除
# SUR 險種貼標為 2 客戶有 9.76% 將被後卡條件排除
# SUR 險種貼標為 3 客戶有 9.43% 將被後卡條件排除
# REI 險種貼標為 1 客戶有 10.63% 將被後卡條件排除
# REI 險種貼標為 2 客戶有 9.92% 將被後卡條件排除
# REI 險種貼標為 3 客戶有 9.56% 將被後卡條件排除
# ACC 險種貼標為 1 客戶有 9.99% 將被後卡條件排除
# ACC 險種貼標為 2 客戶有 9.96% 將被後卡條件排除
# ACC 險種貼標為 3 客戶有 9.73% 將被後卡條件排除
# DD 險種貼標為 1 客戶有 17.19% 將被後卡條件排除
# DD 險種貼標為 2 客戶有 12.35% 將被後卡條件排除
# DD 險種貼標為 3 客戶有 10.87% 將被後卡條件排除
# LTC 險種貼標為 1 客戶有 11.03% 將被後卡條件排除
# LTC 險種貼標為 2 客戶有 10.97% 將被後卡條件排除
# LTC 險種貼標為 3 客戶有 10.45% 將被後卡條件排除
# LIFE 險種貼標為 1 客戶有 13.27% 將被後卡條件排除
# LIFE 險種貼標為 2 客戶有 11.69% 將被後卡條件排除
# LIFE 險種貼標為 3 客戶有 11.76% 將被後卡條件排除

In [ ]:
# 讀取後卡條件客戶與條件列表

## 將需被後卡掉的人，其貼標轉成空值

In [ ]:
df.loc[df.SAS_ID.isin(rm_df.SAS_ID), col] = [np.nan]*len(col)

In [ ]:
# check : 檢查是否有符合後卡條件，但仍有貼標之客戶
miss_cnt = df.loc[df.SAS_ID.isin(rm_df.SAS_ID), col].dropna(how='all').shape[0]
if miss_cnt == 0 :
    print('後卡條件排除成功!!!')
else :
    print('仍有 {} 人有貼標尚未轉換為空值...'.format(miss_cnt))

In [ ]:
# check2 : 檢查是否持有任一貼標客戶人數 = 原人數 - 被排掉人數
# 先抓出有被貼標的人
# tag_index = [i for i in range(len(df)) if any(df.loc[i, col].notna()) ]
tag_index_new = ( df['HOS_MKT_LEVEL'].notna() + df['SUR_MKT_LEVEL'].notna() + df['REI_MKT_LEVEL'].notna() 
             + df['ACC_MKT_LEVEL'].notna() + df['DD_MKT_LEVEL'].notna() + df['LTC_MKT_LEVEL'].notna() 
             + df['LIFE_MKT_LEVEL'].notna() )
df[tag_index_new].shape
# 檢查是否持有任一貼標客戶人數 = 原人數 - 被排掉人數
if (df[tag_index_new].shape[0] == (df_tag_cus.shape[0] - df_tag_cus.SAS_ID.isin(rm_df.SAS_ID).sum())):
    print('後卡條件排除成功!!!', '持有任一貼標客戶數量為 {} 人, 為整體POOL的 {}%'.format(df[tag_index_new].shape[0], round( (df[tag_index_new].shape[0]/len(df)*100) , 2) ))
else :
    print('後卡條件排除失敗!!!')

## 檢查格式

In [ ]:
df.info()

## 存出 後卡後的行銷分大表

In [ ]:
df.to_csv(result_path + 'IFCUSDPDT0001_' + year + month + '_new.csv', index=False)

## 分析歷程與紀錄

### 紀錄時間 : 20220922

**行銷分大表順利後卡完成，但我這邊初步看了一下排除的數字，感覺排除量好像有點多，如下:**
1. 持有任一貼標客戶共379萬 (尚未後卡)
2. 持有任一貼標客戶，後卡後剩337.4萬 (後卡掉41.6萬 --> 好像有點多，約佔10%)

**後續追加分析數字:**
1. 多半由是哪個後卡條件造成 >>> 審規70萬；曾拒保18萬；不適合行銷8萬，推測審歸影響最嚴重，但應該把重點放在各規則的細節，是否有多排除的
2. 檢視【各險種】前10%、11-30%貼標，【各別】失去多少比例的客戶 >>> 完成

**建議/想法:**
1. 看後卡條件細節有沒有可以再精進之處，若時間來得及就調整，若來不及，這次就先上~
2. 個人認為10%有點多，有機會會影響模型行銷潛力判斷，未來可評估是否前卡掉 (待追加分析2)

**短期結論:**
1. 暫不調整細部規則，後續直接觀察後卡排除掉的這些人促約率、拒保率、特承率  (審規需與審查部協作，過往經驗得知需要較長時間合作與調整)

### 紀錄時間 : 20221021

**行銷分大表順利後卡完成，但我這邊初步看了一下排除的數字，感覺排除量好像有點多，如下:**
1. 持有任一貼標客戶共370萬 (尚未後卡)
2. 持有任一貼標客戶，後卡後剩329.6萬 (後卡掉41.8萬 --> 好像有點多，約佔10%)

**後續追加分析數字:**
1. 多半由是哪個後卡條件造成 >>> 審規70萬；曾拒保18萬；不適合行銷8萬，推測審歸影響最嚴重，但應該把重點放在各規則的細節，是否有多排除的
2. 檢視【各險種】前10%、11-30%貼標，【各別】失去多少比例的客戶 >>> 完成

**建議/想法:**
1. 看後卡條件細節有沒有可以再精進之處，若時間來得及就調整，若來不及，這次就先上~
2. 個人認為10%有點多，有機會會影響模型行銷潛力判斷，未來可評估是否前卡掉 (待追加分析2)

**短期結論:**
1. 暫不調整細部規則，後續直接觀察後卡排除掉的這些人促約率、拒保率、特承率  (審規需與審查部協作，過往經驗得知需要較長時間合作與調整)

### 紀錄時間 : 20221121

**行銷分大表順利後卡完成，但我這邊初步看了一下排除的數字，感覺排除量好像有點多，如下:**
1. 持有任一貼標客戶3,986,387 (尚未後卡)
2. 持有任一貼標客戶，後卡後3,551,588 (後卡排434,799)


### 紀錄時間 : 20221220

**行銷分大表順利後卡完成，但我這邊初步看了一下排除的數字，感覺排除量好像有點多，如下:**
1. 持有任一貼標客戶 4,096,407 (尚未後卡)
2. 持有任一貼標客戶，後卡後 3,650,350 (後卡排 446,057)
